In [19]:
from llama_index.vector_stores.qdrant import QdrantVectorStore
from qdrant_client import QdrantClient
from config import *

In [20]:
client = QdrantClient(URL,port=6333)

In [21]:
vector_store = QdrantVectorStore(client=client, collection_name=COLLECTION_NAME)

In [22]:
from llama_index.llms.azure_openai import AzureOpenAI
llm = AzureOpenAI(
    model="gpt-35-turbo",
    deployment_name=deployment_id_gpt4,
    api_key=key,
    azure_endpoint=endpoint,
    api_version=api_version,
)

In [23]:
from llama_index.embeddings.fastembed import FastEmbedEmbedding

embed_model = FastEmbedEmbedding()

Fetching 9 files: 100%|██████████| 9/9 [00:00<00:00, 50466.22it/s]


In [24]:
from llama_index.core import Settings
from llama_index.core  import VectorStoreIndex

Settings.llm = llm
Settings.embed_model = embed_model

In [25]:
index = VectorStoreIndex.from_vector_store(vector_store=vector_store)

In [26]:
query_engine = index.as_query_engine()

In [38]:
query = "What should government do about road safety?"
query = "What is the foreign participation in highways?"

In [39]:
response = query_engine.query(query)

In [40]:
pprint(response.get_formatted_sources())

('> Source (Doc id: 5c78a53f-46df-41af-b6f1-a221bab6f204): — One of the most '
 'attractive destinations for Foreign Direct Investment (FDI). 100% FDI '
 'allowed i...\n'
 '\n'
 '> Source (Doc id: 35dd4a38-1f4d-4b3c-9a9e-3472cf6b50f1): 12 \x02 Report of '
 'the Committee\n'
 'cooperates with insurance companies and the\n'
 'Swedish automotive indus...\n'
 '\n'
 '> Source (Doc id: 78d34def-97dc-4dac-bdb6-080ad865c8b1): to be under taken '
 'henceforth under \n'
 'NHDP. Well laid out policy and regulatory framework, includin...\n'
 '\n'
 '> Source (Doc id: 5f621997-4656-4c8e-bcaa-6d84d83bf688): 10 \x02 Report of '
 'the Committee\n'
 'technical support to state and local governments\n'
 'for constructing, i...\n'
 '\n'
 '> Source (Doc id: f9a8287d-cf93-4ac6-8cdf-b3ae020f533d): NATIONAL HIGHWAYS '
 'DEVELOPMENT PROJECT\n'
 'Infrastructure is critical to improved productivity across ...')


In [29]:

print("query was:", query)
print("answer was:", response)

> Source (Doc id: 888a8a07-aa16-42ae-8d8f-71249e220188): 14  Report of the Committee
of motorization suggest that strategies for
reducing traffic injurie...

> Source (Doc id: 4dd6886a-6874-4f91-9c62-1bb03fbecba3): 8  Report of the Committee
road safety. There is also no effective
mechanism for coordinating th...
query was: What should government do about road safety?
answer was: Governments should make road safety a political priority, develop a multidisciplinary approach to road safety, appoint a lead agency for road safety, give it adequate resources, and make it publicly accountable. They should also set appropriate road safety targets and establish national road safety plans to achieve them, support the creation of safety advocacy groups, create budgets for road safety, and increase investment in demonstrably effective road safety activities.


In [41]:
from pprint import pprint

In [42]:
pprint(str(response))

('According to the excerpt from "Opportunities and Policy Initiatives for '
 'Profitable Partnership and Infrastructure Investment in Indian Highways and '
 'National Highways Development Project: A Comprehensive Overview," '
 'contractors and supervision consultants from around 27 countries are already '
 "in India, helping implement one of the world's largest highways projects. "
 'The excerpt also provides a table listing the countries with contractors and '
 "consultants currently working on India's highways projects. Therefore, "
 'foreign participation in highways in India is significant.')


In [43]:
from llama_index.core.retrievers import VectorIndexRetriever
from llama_index.core.query_engine import RetrieverQueryEngine
from llama_index.core import get_response_synthesizer

In [44]:
# configure retriever
retriever = VectorIndexRetriever(
    index=index,
    similarity_top_k=5,
    llm = llm,
)

In [45]:
# configure response synthesizer
response_synthesizer = get_response_synthesizer(
    response_mode="tree_summarize",
)

In [46]:
# assemble query engine
query_engine = RetrieverQueryEngine(
    retriever=retriever,
    response_synthesizer=response_synthesizer,
)

# query
response = query_engine.query(query)

In [47]:
pprint(str(response))

('According to the excerpt from "Opportunities and Policy Initiatives for '
 'Profitable Partnership and Infrastructure Investment in Indian Highways and '
 'National Highways Development Project: A Comprehensive Overview," '
 'contractors and supervision consultants from around 27 countries are already '
 "in India, helping implement one of the world's largest highways projects. "
 'The excerpt also provides a table that lists the number of contractors and '
 "consultants from each country that are currently working on India's highways "
 'projects. Therefore, foreign participation in highways in India is '
 'significant.')
